Initial tests

In [1]:
import os

# Force CPU fallback for problematic MPS operations
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

from transformers import BertTokenizer, BertModel
import torch.nn as nn

print("Success! BertModel imported")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/kaylahaeusssler/miniforge3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kaylahaeusssler/miniforge3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/kaylahaeusssler/miniforge3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io

Success! BertModel imported


/Users/kaylahaeusssler/miniforge3/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/kaylahaeusssler/miniforge3/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [1]:
import pandas as pd
import transformers
import torch
import torch.nn as nn

# from transformers import BertTokenizer, BertModel
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms
import torchvision.models as models
from PIL import Image

In [1]:
import sys
import torch
import transformers

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

Python: 3.12.6 | packaged by conda-forge | (main, Sep 22 2024, 14:07:06) [Clang 17.0.6 ]
PyTorch: 2.4.0
Transformers: 4.46.3
CUDA available: False


In [2]:
# Try importing with lazy loading disabled
import os

os.environ["TRANSFORMERS_OFFLINE"] = "0"

# Import just the model config first
from transformers import BertConfig

print("BertConfig imported!")

# Then try the model
from transformers.models.bert.modeling_bert import BertModel

print("BertModel imported via direct path!")

BertConfig imported!


: 

In [1]:
# Try importing BertModel by itself
from transformers import BertModel

print("BertModel class imported!")

: 

In [2]:
from transformers import BertTokenizer, BertModel

: 

: 

In [ ]:
# will check and use MPS if available, otherwise CUDA, otherwise CPU
# mps is super fast mac thing
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using device: MPS (Apple)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using device: CUDA (GPU) - {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using device: CPU")

In [2]:
train_subset = pd.read_csv("CheXpert-v1.0-small/train.csv").head()

In [3]:
train_subset.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [4]:
train_subset.columns

Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')

In [5]:
# AP/PA is a view position column, whether Xray facing front or back
# ignoring for now, but maybe we want to keep in the future??
train_subset = train_subset.drop("AP/PA", axis=1)

In [6]:
train_subset = train_subset.fillna(0)

### Generate text reports

In [7]:
def generate_report(row):
    labels = row.iloc[4:]
    findings_list = list(labels[labels == 1].index)
    findings = ", ".join(findings_list)
    return f"X-Ray report findings: {findings}"

In [8]:
test = train_subset.apply(generate_report, axis=1)

In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [16]:
sample_sentence = test[0]

token_ids = []
attention_masks = []

# Encode each review
for report in test:
    batch_encoder = tokenizer.encode_plus(
        report,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

    token_ids.append(batch_encoder["input_ids"])
    attention_masks.append(batch_encoder["attention_mask"])

# Convert token IDs and attention mask lists to PyTorch tensors
token_ids = torch.cat(token_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

In [12]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained(
    "bert-base-uncased",
    output_hidden_states=True,  # Whether the model returns all hidden-states.
)

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [17]:
with torch.no_grad():
    outputs = model(token_ids, attention_mask=attention_masks)
    word_embeddings = outputs.last_hidden_state  # This contains the embeddings

In [26]:
# Get only embedding for CLS token
CLS = word_embeddings[:, 0, :]

In [ ]:
final = nn.Linear(768, 512)

: 

In [28]:
final(CLS).shape

torch.Size([5, 512])

Encoder Classes

### Text Encoder

In [24]:
class TextEncoder(nn.Module):
    def __init__(self, embed_dim=512):
        super(TextEncoder, self).__init__()
        self.bert = BertModel.from_pretrained(
            "bert-base-uncased",
            output_hidden_states=True,
        )
        self.linear = nn.Linear(768, embed_dim)

    def forward(self, token_ids, attention_masks):
        outputs = self.bert(token_ids, attention_mask=attention_masks)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings = self.linear(cls_embeddings)
        # normalizing because we need to compare with image embeddings later
        # for the contrastive similarity
        embeddings = embeddings / embeddings.norm(dim=-1, keepdim=True)
        return embeddings

### Image Encoder

In [25]:
# Use the torchvision's implementation of ResNeXt, but add FC layer to generate 512d embedding.
class VisionEncoder(nn.Module):
    def __init__(self, embed_dim=512):
        super().__init__()
        resnet = models.resnext50_32x4d(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        in_dim = resnet.fc.in_features
        self.proj = nn.Linear(in_dim, embed_dim)

    def forward(self, x):
        features = self.backbone(x)
        features = features.squeeze(-1).squeeze(-1)
        z = self.proj(features)
        # convert to unit vectors for cosine similarity later
        z = z / z.norm(dim=-1, keepdim=True)
        return z

### Dataset

In [26]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        # Text stuff
        self.text_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.reports = df.apply(
            generate_report, axis=1
        )  # Maybe move generate report to inside the dataset idk

        # Vision stuff?
        self.images = df["Path"]
        self.transform = transforms.Compose(
            [
                transforms.Resize((256, 256)),
                transforms.ToTensor(),
            ]
        )

    def __len__(self):
        return len(self.reports)  # This could work or we could do another way

    def __getitem__(self, idx):
        # Text part
        report = self.reports[idx]
        encoder = self.text_tokenizer.encode_plus(
            report,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        # Vision part
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        img_tensor = self.transform(image)

        return {
            "token_ids": encoder["input_ids"],
            "attention_masks": encoder["attention_mask"],
            "image_tensor": img_tensor,
        }  # Is vision part correct?

Samples come in as dataframe and we put in dataset. For each sample, we can then return inputs for text encoder (token ids & attention masks) & for vision encoder (tensor of pixels). We pass relevant inputs to text encoder and vision encoder, get embeddings for that sample. Then blah blah

Training Loop 

In [27]:
def generate_report(row):
    labels = row.iloc[4:]
    findings_list = list(labels[labels == 1].index)
    findings = ", ".join(findings_list)
    return f"X-Ray report findings: {findings}"

In [28]:
train_full = pd.read_csv("CheXpert-v1.0-small/train.csv")

In [29]:
val_full = pd.read_csv("CheXpert-v1.0-small/valid.csv")

In [30]:
# Initialize text encoder & vision encoder models
text_enc = TextEncoder(embed_dim=512)

# Vision
vision_enc = VisionEncoder(embed_dim=512)


# Create train & val datasets
train_dataset = CustomDataset(train_full)
val_dataset = CustomDataset(val_full)

/Users/rishikarandev/miniforge3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/rishikarandev/miniforge3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Generate text embedding
text_enc(
    token_ids=train_dataset[0]["token_ids"],
    attention_masks=train_dataset[0]["attention_masks"],
).shape

torch.Size([1, 512])

In [ ]:
# Generate img embedding
vision_enc(train_dataset[0]["image_tensor"].unsqueeze(0)).shape

torch.Size([1, 512])

In [9]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

### Contrastive Loss

# image_encoder - ResNet or Vision Transformer
# text_encoder - CBOW or Text Transformer
# I[n, h, w, c] - minibatch of aligned images
K: n is batch size, h,w is height width, c is number of channels (3 for RGB)

# T[n, l] - minibatch of aligned texts
K: n is batch size, l is sequence length (number of tokens)

K: aligned means that they are paired up

# W_i[d_i, d_e] - learned proj of image to embed
# W_t[d_t, d_e] - learned proj of text to embed
# t - learned temperature parameter

# extract feature representations of each modality
I_f = image_encoder(I) #[n, d_i] 
this happens in vision_enc
K: get features from resnet

T_f = text_encoder(T) #[n, d_t]
this happens in text_enc
K: get features from bert

# joint multimodal embedding [n, d_e]
K: this is projecting to joint embedding space
I_e = l2_normalize(np.dot(I_f, W_i), axis=1)
T_e = l2_normalize(np.dot(T_f, W_t), axis=1)

# scaled pairwise cosine similarities [n, n]
logits = np.dot(I_e, T_e.T) * np.exp(t) #K: scale by temperature


# symmetric loss function
labels = np.arange(n) #K: correct match is on the diagonal
# K: now we compute symmetric loss
loss_i = cross_entropy_loss(logits, labels, axis=0) #K: image to text
loss_t = cross_entropy_loss(logits, labels, axis=1) #K: text to image
# K: now average in both directions
loss = (loss_i + loss_t)/2

In [ ]:
# a lot of the steps from the pseudo code are already done in other places!! which is lit
# I_f, T_f, I_e, T_e are all done in other places
def contrastive_loss(image_embeddings, text_embeddings, temperature=0.07):
    # we can make the t a fixed parameter but most CLIP reproductions make it fixed
    # so just keeping it as argument for now

    batch_size = image_embeddings.shape[0]  # called n in the pseudo code

    # we already have normalized embeddings, so we can skip to cosine similarity part of pseudo code
    # "scaled pairwise cosine similarities [n, n]"
    # dividing by temperature to scale the logits
    logits = torch.matmul(image_embeddings, text_embeddings.T) / temperature
    # now "symmetric loss function"
    labels = torch.arange(batch_size, device=image_embeddings.device)
    loss_img_to_text = nn.CrossEntropyLoss()(logits, labels)
    loss_text_to_img = nn.CrossEntropyLoss()(logits.T, labels)

    # average loss in both directions
    loss = (loss_img_to_text + loss_text_to_img) / 2

    return loss

In [ ]:
# training setup, referencing homework code
lr = 1e-3  # maybe we will adjust this
batch_size = 32  # already set in the DataLoader
temperature = 0.07
max_epoch_number = 20
test_freq = 50

# move models to device
text_enc = text_enc.to(device)
vision_enc = vision_enc.to(device)

# combine parameters of both models for the optimizer
# our homework only used one model, so now we gotta grab both model params
all_parameters = list(text_enc.parameters()) + list(vision_enc.parameters())
optimizer = torch.optim.Adam(all_parameters, lr=lr)


epoch = 0
iteration = 0
while True:
    text_enc.train()
    vision_enc.train()

    batch_losses = []

    for batch in train_loader:
        images = batch["image_tensor"].to(device)
        token_ids = batch["token_ids"].squeeze(1).to(device)  # Remove extra dimension
        attention_masks = batch["attention_masks"].squeeze(1).to(device)

        optimizer.zero_grad()

        # this is forward pass
        image_embeddings = vision_enc(images)
        text_embeddings = text_enc(token_ids, attention_masks)

        # get loss
        loss = contrastive_loss(image_embeddings, text_embeddings, temperature)

        # now backward pass
        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()

        batch_losses.append(batch_loss_value)

        # i had Claude rewrite this validation loss part
        if iteration % test_freq == 0:
            text_enc.eval()
            vision_enc.eval()

            val_losses = []  # Track validation losses

            with torch.no_grad():
                for batch_val in val_loader:
                    # Get validation data
                    val_images = batch_val["image_tensor"].to(device)
                    val_token_ids = batch_val["token_ids"].squeeze(1).to(device)
                    val_attention_masks = (
                        batch_val["attention_masks"].squeeze(1).to(device)
                    )

                    # Forward pass
                    val_img_embeds = vision_enc(val_images)
                    val_txt_embeds = text_enc(val_token_ids, val_attention_masks)

                    # Compute loss
                    val_loss = contrastive_loss(
                        val_img_embeds, val_txt_embeds, temperature
                    )
                    val_losses.append(val_loss.item())

            # Print validation results
            avg_val_loss = np.mean(val_losses)
            print(f"epoch:{epoch:2d} iter:{iteration:3d} val: loss:{avg_val_loss:.3f}")

            # Back to training mode
            text_enc.train()
            vision_enc.train()
        iteration += 1

    train_loss = np.mean(batch_losses)
    print(f"epoch:{epoch:2d} iter:{iteration:4d} train loss:{train_loss:.3f}\n")

    epoch += 1
    if max_epoch_number < epoch:
        break

In [ ]:
# Ignore for now just copied from our hw
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
batch_size = 32
criterion = nn.BCELoss()
max_epoch_number = 20
test_freq = 50

model = model.to(device)
epoch = 0
iteration = 0
while True:
    batch_losses = []
    for imgs, targets in train_loader:
        imgs = imgs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()

        model_result = model(imgs)
        loss = criterion(model_result, targets.type(torch.float))

        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()

        batch_losses.append(batch_loss_value)

        if iteration % test_freq == 0:
            model.eval()
            with torch.no_grad():
                model_result = []
                targets = []
                for imgs, batch_targets in val_loader:
                    imgs = imgs.to(device)
                    batch_targets = batch_targets.to(device)
                    model_batch_result = model(imgs)
                    # model_pred_binary = (torch.sigmoid(model_batch_result) > 0.5).int().cpu().numpy()
                    model_result.extend(model_batch_result.cpu().numpy())
                    targets.extend(batch_targets.cpu().numpy())

            result = calculate_metrics(np.array(model_result), np.array(targets))
            print(
                "epoch:{:2d} iter:{:3d} test: "
                "micro f1: {:.3f} "
                "macro f1: {:.3f} "
                "samples f1: {:.3f}".format(
                    epoch,
                    iteration,
                    result["micro/f1"],
                    result["macro/f1"],
                    result["samples/f1"],
                )
            )

            model.train()
        iteration += 1

    loss_value = np.mean(batch_losses)
    print(
        "epoch:{:2d} iter:{:3d} train: loss:{:.3f}".format(epoch, iteration, loss_value)
    )
    # if epoch % save_freq == 0:
    #    checkpoint_save(model, save_path, epoch)
    epoch += 1
    if max_epoch_number < epoch:
        break